# BUS 32100, Week 4
# In-class exercise: data wrangling with `Pandas` using MTA data

Learning objectives:

* Practice combining datasets into a single dataframe
* Understand basic methods for data quality checking and cleaning 
* Use visualization as an aid in exploring data quality
* Practice working with timeseries data and plotting timeseries data

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

## Download the data and import into `Pandas`

For this week's exercise, you must download the data yourself from the source! You will need to download multiple files and append them together into a single DataFrame. 

* Turnstile data for the NYC subway is available here: http://web.mta.info/developers/turnstile.html
* Each file contains one week of data. You should download **at least three files of data** (though you can download more). A good starting point is to download these three files: 
    * http://web.mta.info/developers/data/nyct/turnstile/turnstile_160903.txt
    * http://web.mta.info/developers/data/nyct/turnstile/turnstile_160910.txt
    * http://web.mta.info/developers/data/nyct/turnstile/turnstile_160917.txt

## Data dictionary

[Refer to the data dictionary for definitions of each column.](http://web.mta.info/developers/resources/nyct/turnstile/ts_Field_Description.txt)

## Two options for downloading the data

1. Copy/paste the URLs above into a browser and save each as a txt file. Import each txt file separately into Pandas. Use what we learned about concatenating files to append the three files into one `DataFrame`.
2. The more advanced approach would be to download multiple files using Python/Pandas at one time, without saving each txt to your local computer. Hint: Notice that the URLs are similar, except for the date. The URL is "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt", where the {} changes for each file's date.

**If you want to try option #2 but need some hints to start, see below.**

## Optional: hints for automating the download of multiple files

1. Create a list of dates (e.g. 160903, 160910, etc). You can either make the list manually or somehow automate that, too. 
2. Write a for-loop over the list of dates that takes the general URL and fills in the {} with each date for each iteration (look into [Python `f strings`](https://realpython.com/python-f-strings/) to help with filling in the {} in the string).
3. Find out how to pass a URL to Pandas instead of a local csv to get back a DataFrame.
4. How can you keep track of all the DF's you're creating? Can you put them in some sort of container until you're ready to concatenate them all?
5. After you've looped over all the dates you want to use, your DF's should be in some container. What method takes a container (e.g. list) of DFs and returns them all concatenated to each other?

## Your tasks tonight: 

1. Check the column names of the data: Do they seem correct? How can you print just the column names of a df? If you notice an error in a column name, how can you correct it? Look into the `strip()` method if needed. 

2. We'll eventually want to plot a time series of turnstile entries/exits. What do we need to do to the data to make a timeseries? Is there a way to tell Pandas that a particular column has datatime data as you're importing it, so you don't need to cast it later? Look into this! But casting the column after you import the file is ok, too.

3. There may be some errors in the data. For example, there are some timestamps that are audited/updated. Can you use the data dictionary, combined with EDA, to figure out where these rows are? Do you need to use the entire datetime for this step, or will you need to drop a portion of it? For example, do you need to use the minutes/seconds, or can you drop that from the datetime? You can't delete minutes/seconds from a datetime, but you can set them to zero without changing the day/month/year. Look into it. 

3. Look into the `DESc` column: what does it represent? How does this look in the data? Do you need to remove any data to account for what this column is doing?

4. What is the `entries` column actually counting? Is `entires` the number of people going through a turnstile every hour? How can we change this data to get the number of entries at a given time? This one is **tricky**! Look into `shift` and `apply` and see if you can figure out how to move a number from the next/previous day to be on the same row as today. That is, how can I get yesteday's `entries` on the same row as today so I can subtract them to get the number of people who went through the turnstile today? 

4. It seems like we're counting cumulative entries. But is the counter always working correctly? Does it ever go wrong? Does it ever run backwards, perhaps? 

5. Use some plots to quality check the data. Some suggestions: histogram of daily entries/exits; bar chart of the top 10 stations; box and whisker plot of entries per station to find outliers; whatever else you can think of! If you have an idea of what to make but don't know how to do it in Python, ask us :)

In [52]:
df1 = pd.read_csv('turnstile_160903.txt', delimiter='\t')  # adjust delimiter as needed
df2 = pd.read_csv('turnstile_160910.txt', delimiter='\t')
df3 = pd.read_csv('turnstile_160917.txt', delimiter='\t')

# Append the three DataFrames into one
combined_df = pd.concat([df1, df2, df3], ignore_index=True)

print(combined_df)

       C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS                                                               
0       A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016...                                                                                
1       A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016...                                                                                
2       A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016...                                                                                
3       A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016...                                                                                
4       A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016...                                                                                
...                                                   ...                                                                                
580890  TRAM2,R469,00-05-01,RIT-RO

In [53]:
combined_df.columns

Index(['C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS                                                               '], dtype='object')

The columns are not divided, so I am going to strip it 

In [54]:
headers = ['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME', 'DESC', 'ENTRIES', 'EXITS']

# Split the column by commas and expand into separate columns
combined_df = combined_df[combined_df.columns[0]].str.split(',', expand=True)

# Assign the headers to the newly created columns
combined_df.columns = headers

print(combined_df.head())


    C/A  UNIT       SCP STATION LINENAME DIVISION        DATE      TIME  \
0  A002  R051  02-00-00   59 ST   NQR456      BMT  08/27/2016  00:00:00   
1  A002  R051  02-00-00   59 ST   NQR456      BMT  08/27/2016  04:00:00   
2  A002  R051  02-00-00   59 ST   NQR456      BMT  08/27/2016  08:00:00   
3  A002  R051  02-00-00   59 ST   NQR456      BMT  08/27/2016  12:00:00   
4  A002  R051  02-00-00   59 ST   NQR456      BMT  08/27/2016  16:00:00   

      DESC     ENTRIES                                              EXITS  
0  REGULAR  0005799442  0001966041                                    ...  
1  REGULAR  0005799463  0001966044                                    ...  
2  REGULAR  0005799492  0001966079                                    ...  
3  REGULAR  0005799610  0001966155                                    ...  
4  REGULAR  0005799833  0001966214                                    ...  


In [55]:
combined_df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [56]:
combined_df.sample(10)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
111427,PTH05,R543,00-04-00,EXCHANGE PLACE,1,PTH,08/31/2016,18:20:14,REGULAR,0000113111,0000010950 ...
471033,N333A,R141,00-00-01,FOREST HILLS 71,EFMR,IND,09/13/2016,01:00:00,REGULAR,0000008376,0000012053
159419,R304,R206,00-00-00,125 ST,23,IRT,08/30/2016,17:00:00,REGULAR,1693235410,0788868125 ...
495225,N606,R025,00-00-04,JAMAICA CENTER,EJZ,IND,09/13/2016,16:00:00,REGULAR,0011875940,0007860625
491637,N553,R422,00-00-01,BAY PKWY,F,IND,09/13/2016,13:00:00,REGULAR,0002168321,0000838646 ...
105538,N601,R319,00-00-02,LEXINGTON AV/63,F,IND,08/29/2016,12:00:00,REGULAR,0016246695,0011630029 ...
306089,PTH07,R550,00-00-07,CITY / BUS,1,PTH,09/03/2016,04:35:54,REGULAR,0001162031,0001488988 ...
299815,N603,R303,00-00-03,21 ST-QNSBRIDGE,F,IND,09/05/2016,04:00:00,REGULAR,0012155161,0007178214 ...
263524,N205,R195,02-00-01,161/YANKEE STAD,BD4,IND,09/03/2016,16:22:00,REGULAR,0003785736,0004372087
98355,N521,R300,01-06-02,2 AV,F,IND,08/29/2016,08:00:00,REGULAR,0000176029,0000027298 ...


2. We'll eventually want to plot a time series of turnstile entries/exits. What do we need to do to the data to make a timeseries? Is there a way to tell Pandas that a particular column has datatime data as you're importing it, so you don't need to cast it later? Look into this! But casting the column after you import the file is ok, too

In [57]:
combined_df['DATE'] = pd.to_datetime(combined_df['DATE'], format='%m/%d/%Y')

# Set 'date' column as the index
combined_df.set_index('DATE', inplace=True)

# Optional: Sort the DataFrame by the index (if needed)
combined_df.sort_index(inplace=True)


In [60]:
combined_df.head(50)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,TIME,DESC,ENTRIES,EXITS
DATE,,,,,,,,,,
2016-08-27,A002,R051,02-00-00,59 ST,NQR456,BMT,00:00:00,REGULAR,0005799442,0001966041 ...
2016-08-27,N325A,R218,00-03-01,ELMHURST AV,MR,IND,00:00:00,REGULAR,0000229111,0000259310 ...
2016-08-27,N325A,R218,00-03-01,ELMHURST AV,MR,IND,04:00:00,REGULAR,0000229111,0000259319 ...
2016-08-27,N325A,R218,00-03-01,ELMHURST AV,MR,IND,08:00:00,REGULAR,0000229137,0000259324 ...
2016-08-27,N325A,R218,00-03-01,ELMHURST AV,MR,IND,12:00:00,REGULAR,0000229248,0000259335 ...
2016-08-27,N325A,R218,00-03-01,ELMHURST AV,MR,IND,16:00:00,REGULAR,0000229343,0000259356 ...
2016-08-27,N325A,R218,00-03-01,ELMHURST AV,MR,IND,20:00:00,REGULAR,0000229417,0000259383 ...
2016-08-27,N325A,R218,00-03-02,ELMHURST AV,MR,IND,00:00:00,REGULAR,0000220423,0000286836 ...
2016-08-27,N325A,R218,00-03-02,ELMHURST AV,MR,IND,04:00:00,REGULAR,0000220432,0000286857 ...


3. There may be some errors in the data. For example, there are some timestamps that are audited/updated. Can you use the data dictionary, combined with EDA, to figure out where these rows are? Do you need to use the entire datetime for this step, or will you need to drop a portion of it? For example, do you need to use the minutes/seconds, or can you drop that from the datetime? You can't delete minutes/seconds from a datetime, but you can set them to zero without changing the day/month/year. Look into it.

In [ ]:
if same UNIT/ same day, max Entry- min Entry 

In [37]:
columns_to_replace = ['ENTRIES', 'EXITS']  # replace with desired column names
combined_df[columns_to_replace] = combined_df[columns_to_replace].replace(0, np.nan)
combined_df.isna().sum()

C/A         0
UNIT        0
SCP         0
STATION     0
LINENAME    0
DIVISION    0
DATE        0
TIME        0
DESC        0
ENTRIES     0
EXITS       0
dtype: int64

4. Look into the `DESc` column: what does it represent? How does this look in the data? Do you need to remove any data to account for what this column is doing?


In [ ]:
present the "REGULAR" scheduled audit event (Normally occurs every 4 hours)
           1. Audits may occur more that 4 hours due to planning, or troubleshooting activities. 
           2. Additionally, there may be a "RECOVR AUD" entry: This refers to a missed audit that was recovered

5. What is the `entries` column actually counting? Is `entires` the number of people going through a turnstile every hour? How can we change this data to get the number of entries at a given time? This one is **tricky**! Look into `shift` and `apply` and see if you can figure out how to move a number from the next/previous day to be on the same row as today. That is, how can I get yesteday's `entries` on the same row as today so I can subtract them to get the number of people who went through the turnstile today? 

In [63]:
grouped = combined_df.groupby(['UNIT', 'DATE'])['ENTRIES'].agg([max, min])

# Step 3: Calculate the difference and create a new column
grouped['max_min_difference'] = grouped[max] - grouped[min]

# Step 4: Merge the result back to the original DataFrame
combined_df = combined_df.merge(grouped['max_min_difference'], on=['UNIT', 'DATE'], how='left')

/tmp/ipykernel_417/3556948290.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  grouped = combined_df.groupby(['UNIT', 'DATE'])['ENTRIES'].agg([max, min])
/tmp/ipykernel_417/3556948290.py:1: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  grouped = combined_df.groupby(['UNIT', 'DATE'])['ENTRIES'].agg([max, min])


TypeError: unsupported operand type(s) for -: 'str' and 'str'

6. Use some plots to quality check the data. Some suggestions: histogram of daily entries/exits; bar chart of the top 10 stations; box and whisker plot of entries per station to find outliers; whatever else you can think of! If you have an idea of what to make but don't know how to do it in Python, ask us :)

In [ ]:
#histogram of daily entries 
plt.barh(x_variable, y_variable) 

In [45]:
#bar chart of the top 10 stations 
station_totals = combined_df.groupby('STATION')['ENTRIES'].sum()

# Sort the stations by total entries in descending order and get the top 10
top_10_stations = station_totals.sort_values(ascending=False).head(10)

top_10_stations.plot(station_totals, kind='bar', color='skyblue', figsize=(10, 6))

plt.bar(top_10_stations, ENTRIES) 
plt.title('Top 10 Stations by Total Entries')
plt.xlabel('Station')
plt.ylabel('Total Entries')
plt.show()

#plt.bar(x_variable, y_variable) 





TypeError: `Series.plot()` should not be called with positional arguments, only keyword arguments. The order of positional arguments will change in the future. Use `Series.plot(kind=STATION
1 AV               0011448160001144826300114483620011449527001145...
103 ST             0012085635001208565200120858520012086340001208...
103 ST-CORONA      0013979193001397923300139795600013979913001398...
104 ST             0005413207000541323000054133910005413583000541...
110 ST             0003511361000351139000035114440003511600000351...
                                         ...                        
WOODHAVEN BLVD     0004071820000407183200040719290004072056000407...
WOODLAWN           0006459312000645935400064595880006459953000646...
WORLD TRADE CTR    0001204229000120424300012042670001204389000120...
YORK ST            0011870760001187080500118709210011871329001187...
ZEREGA AV          0053088150005308815600530881800053088223005308...
Name: ENTRIES, Length: 374, dtype: object)` instead